In [52]:
#!pip install pesq auraloss torchmetrics

In [53]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
import torch.nn as nn
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np
import librosa
import soundfile as sf
from PIL import Image
import torchvision.transforms as T
import numpy.fft as fft
import torch.nn.functional as F

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [54]:
def spectral_centroid(x):
    # Last axis is always channels
    num_channels = x.shape[-1]
    centroids = []

    for c in range(num_channels):
        data_c = x[..., c]  # Extract channel c
        ndim = data_c.ndim

        if ndim == 1:  # Audio single signal
            spectrum = np.abs(np.fft.rfft(data_c))
            freq_bins = np.fft.rfftfreq(data_c.shape[-1], d=1)
            weighted_sum = np.sum(spectrum * freq_bins)
            sum_of_weights = np.sum(spectrum)
            centroid = weighted_sum / sum_of_weights if sum_of_weights != 0 else 0

        elif ndim == 2:  # Image (H, W)
            spectrum = np.abs(np.fft.rfft(data_c, axis=1))
            freq_bins = np.fft.rfftfreq(data_c.shape[1], d=1)
            weighted_sum = np.sum(spectrum * freq_bins, axis=1)
            sum_of_weights = np.sum(spectrum, axis=1)
            row_centroids = np.divide(weighted_sum, sum_of_weights, out=np.zeros_like(weighted_sum), where=sum_of_weights != 0)
            centroid = np.mean(row_centroids)

        elif ndim == 3:  # 3D data (H, W, D)
            spectrum = np.abs(np.fft.rfft(data_c, axis=2))
            freq_bins = np.fft.rfftfreq(data_c.shape[2], d=1)
            weighted_sum = np.sum(spectrum * freq_bins, axis=2)
            sum_of_weights = np.sum(spectrum, axis=2)
            slice_centroids = np.divide(weighted_sum, sum_of_weights, out=np.zeros_like(weighted_sum), where=sum_of_weights != 0)
            centroid = np.mean(slice_centroids)

        else:
            raise ValueError("Unsupported data dimensionality after removing channel axis.")

        centroids.append(centroid)

    # Average across channels and normalize to [0, 1]
    return (np.mean(centroids) * 2) / num_channels

#############################################  audio related  #############################################
def set_audio_target(device, file_path, max_samples):
    waveform, sr = wav_to_tensor(file_path)
    waveform /= torch.max(torch.abs(waveform))  # normalize to [-1, 1]
    waveform = waveform.to(device)
    n_samples = min(max_samples, waveform.shape[0])

    return waveform[:n_samples].view(-1,1), n_samples, sr


def wav_to_tensor(filepath, sr=None, mono=True):
    audio, sample_rate = librosa.load(filepath, sr=sr, mono=mono)
    tensor = torch.tensor(audio)
    return tensor, sample_rate


def tensor_to_wav(tensor, sample_rate, output_filepath='sample.wav'):
    audio = tensor.to('cpu').detach().numpy()
    sf.write(output_filepath, audio, sample_rate)

#############################################  network related  #############################################
@torch.jit.script
def sine_block(x: torch.Tensor, w0: float, a0: float):
    return a0 * torch.sin(w0 * x)

class SineLayer(nn.Module):
    def __init__(self, w0, A=1.0):
        super().__init__()
        self.a0 = A
        self.w0 = w0

    def forward(self,x):
        return sine_block(x, self.w0, self.a0)

#############################################  training related  #############################################
def batched_forward(model, coords, batch_size=65536):
    """
    Perform <> in smaller batches to avoid OOM.
    """
    outputs = []
    N = coords.shape[0]
    for i in range(0, N, batch_size):
        coords_batch = coords[i:i+batch_size]
        out = model(coords_batch)
        outputs.append(out.detach().cpu())
    return torch.cat(outputs, dim=0)


def train(model, coords, ground_truth, config, device, nb_epochs, batch_size):
    model.to(device)
    coords = coords.to(device)
    ground_truth = ground_truth.to(device)

    loss_fun = nn.MSELoss()
    model_optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    scheduler = StepLR(model_optimizer, step_size=config.scheculer_step, gamma=config.gamma)

    psnr_history = []
    total_points = coords.shape[0]
    best_psnr = -float('inf')
    best_model_state = None

    pbar = tqdm(range(nb_epochs))
    for _ in pbar:
        indices = torch.randperm(total_points)
        epoch_loss = 0.0

        for i in range(0, total_points, batch_size):
            batch_indices = indices[i:i + batch_size]
            coords_batch = coords[batch_indices, :]
            gt_batch = ground_truth[batch_indices, :]

            output = model(coords_batch)
            loss = ((output - gt_batch) ** 2).mean()
            epoch_psnr = 20 * np.log10(1.0 / np.sqrt(loss.item()))

            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()

            epoch_loss += loss.item()

        # Full-batch PSNR estimate (optional but used in your original logic)
        epoch_psnr = 20 * np.log10(1.0 / np.sqrt(loss.item()))
        psnr_history.append(epoch_psnr)

        if epoch_psnr > best_psnr:
            best_psnr = epoch_psnr
            best_model_state = model.state_dict()

        pbar.set_postfix({'PSNR': f'{epoch_psnr:.2f} dB'})
        scheduler.step()

    # Restore best model and compute its output
    model.load_state_dict(best_model_state)
    output = batched_forward(model, coords, batch_size=65536)

    return psnr_history, output

In [55]:
# define the models

# Original SIREN
class SIREN(nn.Module):
    def __init__(self, in_dim, HL_dim, out_dim, w0=30, first_w0=3000, n_HLs=4):
        super().__init__()
        self.net = []

        self.net.append(nn.Linear(in_dim, HL_dim))
        self.net.append(SineLayer(first_w0))
        for _ in range(n_HLs-1):
            self.net.append(nn.Linear(HL_dim, HL_dim))
            self.net.append(SineLayer(w0))
        self.net.append(nn.Linear(HL_dim, out_dim))

        self.net = nn.Sequential(*self.net)

        # init weights
        with torch.no_grad():
            self.net[0].weight.uniform_(-1.0/in_dim, 1.0/in_dim)
            for i in range(n_HLs):
                self.net[(i+1)*2].weight.uniform_(-np.sqrt(6.0/HL_dim)/w0, np.sqrt(6.0/HL_dim)/w0)

    def forward(self, x):
        return self.net(x)


class SIREN_square(nn.Module):
    def __init__(self, omega_0=30, in_dim=1, HL_dim=256, out_dim=1, first_omega=30, n_HLs=4, spectral_centeroid = 0, S0=0, S1=0):
        super().__init__()
        self.in_dim = in_dim
        self.omega_0 = omega_0
        self.S0 = S0    # noise scale between INPUT and 1st HIDDEN LAYER
        self.S1 = S1    # noise scale between 1st and 2nd HIDDEN LAYERS
        self.SC = spectral_centeroid
        self.n_ch = out_dim

        # define network architecture
        self.net = []
        self.net.append(nn.Linear(in_dim, HL_dim))
        self.net.append(SineLayer(first_omega))
        for _ in range(n_HLs-1):
            self.net.append(nn.Linear(HL_dim, HL_dim))
            self.net.append(SineLayer(omega_0))
        self.net.append(nn.Linear(HL_dim, out_dim))

        self.net = nn.Sequential(*self.net)

        # initialize weights
        with torch.no_grad():
            self.net[0].weight.uniform_(-1.0/in_dim, 1.0/in_dim)

            for i in range(n_HLs):
                self.net[(i+1)*2].weight.uniform_(-np.sqrt(6.0/HL_dim)/self.omega_0, np.sqrt(6.0/HL_dim)/self.omega_0)

            self.weights0 = self.net[0].weight.detach().clone()
            self.weights2 = self.net[2].weight.detach().clone()

        # add noise
        self.set_noise_scales()
        self.add_noise()

    def set_noise_scales(self):
        # insert the emperical formula to set S0 and S1 values as a function of (SC/n_ch)
        if self.in_dim == 1:    # audio
            a, b = 7, 3
            self.S0 = 3500*(1-np.exp(-a*self.SC/self.n_ch))
            self.S1 = self.SC/self.n_ch * b
        #elif self.in_dim == 2:   images
            #a, b = 5, 0.4
            #self.S0 = 50*(1-np.exp(-a*self.SC/self.n_ch))
            #self.S1 = self.SC/self.n_ch * b
        #elif self.in_dim == 3:  # 3D
            #self.S0 = self.S0
            #self.S1 = self.S1

        # or manually set the noise scales
        # self.S0 = self.S0
        # self.S1 = self.S1

        # print(f'spectral centeroid = {self.SC}, SIREN^2 set to noise scales S0={self.S0} and S1={self.S1}')

    def add_noise(self):
        with torch.no_grad():
            # INPUT LAYER --> FIRST HIDDEN LAYER (uniform + noise)
            scale = self.S0 / self.omega_0
            self.net[0].weight.copy_(self.weights0 + torch.randn_like(self.weights0) * scale)

            # FIRST --> SECOND HIDDEN LAYER (uniform + noise)
            scale = self.S1 / self.omega_0
            self.net[2].weight.copy_(self.weights2 + torch.randn_like(self.weights2) * scale)

            # Free memory after use
            del self.weights0, self.weights2
            torch.cuda.empty_cache()

    def forward(self, coords):
        return self.net(coords)


In [56]:
from pesq import pesq
from torchmetrics.functional.audio import signal_noise_ratio

def calculate_lsd(y_pred, y_true, sr=16000):
    if not torch.is_tensor(y_pred): y_pred = torch.tensor(y_pred)
    if not torch.is_tensor(y_true): y_true = torch.tensor(y_true)

    device = y_true.device
    y_pred = y_pred.to(device)

    n_fft = 2048
    hop_length = 512
    window = torch.hann_window(n_fft).to(device)

    stft_pred = torch.stft(
        y_pred.view(-1), n_fft=n_fft, hop_length=hop_length,
        return_complex=True, window=window
    )
    stft_true = torch.stft(
        y_true.view(-1), n_fft=n_fft, hop_length=hop_length,
        return_complex=True, window=window
    )

    # Power spectra
    spec_pred = torch.abs(stft_pred) ** 2
    spec_true = torch.abs(stft_true) ** 2

    # -------- Silence masking (stricter) --------
    eps = 1e-10
    frame_energy = torch.sum(spec_true, dim=0)
    frame_energy_db = 10 * torch.log10(frame_energy + eps)

    mask = frame_energy_db > (frame_energy_db.max() - 60)
    if mask.sum() == 0:
        return 0.0

    spec_pred = spec_pred[:, mask]
    spec_true = spec_true[:, mask]

    # -------- Frequency restriction (speech standard) --------
    max_freq_bin = int((8000 / sr) * (n_fft // 2 + 1))
    spec_pred = spec_pred[:max_freq_bin]
    spec_true = spec_true[:max_freq_bin]

    # -------- Proper LSD (log of ratio) --------
    spec_pred = torch.clamp(spec_pred, min=eps)
    spec_true = torch.clamp(spec_true, min=eps)

    log_ratio = torch.log10(spec_true / spec_pred)
    lsd = torch.mean(torch.sqrt(torch.mean(log_ratio ** 2, dim=0)))

    return lsd.item()


def calculate_metrics(y_pred, y_true, sr):
    """
    Calculates PSNR, SNR, LSD, and PESQ.
    y_pred, y_true: Torch tensors or numpy arrays (shape: [samples] or [1, samples])
    sr: sampling rate
    """
    # 1. Prepare Data (Move to CPU, numpy, flatten)
    if torch.is_tensor(y_pred): y_pred_np = y_pred.detach().cpu().numpy().flatten()
    else: y_pred_np = y_pred.flatten()

    if torch.is_tensor(y_true): y_true_np = y_true.detach().cpu().numpy().flatten()
    else: y_true_np = y_true.flatten()

    # Ensure tensors for TorchMetrics
    t_pred = torch.tensor(y_pred_np).float()
    t_true = torch.tensor(y_true_np).float()

    # --- Metric 1: PSNR (Peak Signal-to-Noise Ratio) ---
    # Higher is better
    mse = np.mean((y_pred_np - y_true_np) ** 2)
    max_val = 2.0 # Since audio is in [-1, 1], range is 2
    if mse == 0: psnr = 100.0
    else: psnr = 20 * np.log10(max_val / np.sqrt(mse))

    # --- Metric 2: SNR (Signal-to-Noise Ratio) ---
    # Higher is better
    # SNR = 10 * log10(Signal_Power / Noise_Power)
    noise_power = np.mean((y_pred_np - y_true_np) ** 2)
    signal_power = np.mean(y_true_np ** 2)
    if noise_power == 0: snr = 100.0
    else: snr = 10 * np.log10(signal_power / noise_power)

    # --- Metric 3: LSD (Log-Spectral Distance) ---
    # Lower is better (0 is perfect)
    # Measures distance between spectrograms (captures blurry high-freqs)
    lsd = calculate_lsd(t_pred, t_true, sr=sr)

    # --- Metric 4: PESQ (Perceptual Evaluation of Speech Quality) ---
    # Range [-0.5 to 4.5]. Higher is better.
    # Replaces ViSQOL as the standard "perceptual" metric.
    # Note: PESQ requires 16k or 8k. We resample if needed.
    try:
        if sr != 16000:
            # Simple resampling for PESQ calculation
            t_pred_16k = librosa.resample(y_pred_np, orig_sr=sr, target_sr=16000)
            t_true_16k = librosa.resample(y_true_np, orig_sr=sr, target_sr=16000)
            pesq_score = pesq(16000, t_true_16k, t_pred_16k, 'wb') # 'wb' = Wideband
        else:
            pesq_score = pesq(sr, y_true_np, y_pred_np, 'wb')
    except Exception as e:
        pesq_score = 0.0
        print(f"PESQ Error: {e}")

    return {"PSNR": psnr, "SNR": snr, "LSD": lsd, "PESQ": pesq_score}

In [57]:
import scipy.io.wavfile as wavfile
import io
from IPython.display import Audio, display

class config:
    lr = 1e-4               # learning rate
    gamma = 0.99            # learning rate decay factor
    scheculer_step = 20     # iteration frequency to update learning rate
    n_HLs = 4               # number of hidden layers
    nb_epochs = 500         # total training epochs
    HL_dim = 222            # hidden layer dimension
    max_samples = 150000
    n_channels = 1

def get_networks(config, n_channels, HL_dim=256, SC=0, S0=0, S1=0):
    return [
        SIREN(in_dim=1, HL_dim=HL_dim, out_dim=n_channels, first_w0=3000, w0=30, n_HLs=config.n_HLs).to(device),
        SIREN_square(omega_0=30, in_dim=1, HL_dim=HL_dim, out_dim=1, first_omega=30, n_HLs=config.n_HLs, spectral_centeroid = SC, S0=S0, S1=S1).to(device),
    ]

audio_files = ['the_rainbow.wav','military_action.wav','there_was_great_support.wav']

# display the ground truth audio files
for i in range(len(audio_files)):
  print(f'Ground truth: {audio_files[i]}')
  waveform_gt, _, sample_rate = set_audio_target(device, audio_files[i], config.max_samples)
  display(Audio(waveform_gt.detach().cpu().squeeze().numpy(), rate=sample_rate))

Ground truth: the_rainbow.wav


Ground truth: military_action.wav


Ground truth: there_was_great_support.wav


In [58]:
# --- EXPERIMENT CONFIGURATION ---
SR_FACTOR = 2  # 2 = Train on 8kHz (predict 16kHz). 4 = Train on 4kHz.

for file in audio_files:
    print(f' PROCESSING: {file}')

    # 1. Load Ground Truth (High Res)
    waveform_gt, n_samples, sample_rate = set_audio_target(device, file, config.max_samples)

    # 2. Create Full Coordinate Grid (Target)
    coords_full = torch.linspace(-1, 1, n_samples, device=device).view(-1, config.n_channels)

    # EXPERIMENT 1: AUDIO RECONSTRUCTION (FITTING)
    # Params: Aggressive Noise (S0=3000, S1=1.0) to capture high frequencies.
    print(f"\n[Task 1] Audio Representation (Fitting)")

    # Calculate SC from the full target
    SC_fit = spectral_centroid(waveform_gt.detach().cpu().numpy())

    # Initialize FRESH models for Fitting with Standard WINNER params
    models_fit = get_networks(config, n_channels=config.n_channels, HL_dim=config.HL_dim, SC=SC_fit, S0=3000, S1=1.0)

    for model in models_fit:
        model_name = f"{model.__class__.__name__}"
        print(f"  > Training {model_name} on Full High-Res data...")

        # Train on FULL data
        batch_size = min(n_samples, 512*512)
        train(model, coords_full, waveform_gt, config, device, nb_epochs=config.nb_epochs, batch_size=batch_size)

        # Inference (Full)
        with torch.no_grad():
            output_fit = batched_forward(model, coords_full, batch_size=65536).to(device)

        # Metrics
        scores = calculate_metrics(output_fit, waveform_gt, sample_rate)
        print(f"    {model_name} Fitting -> PSNR: {scores['PSNR']:.2f} | LSD: {scores['LSD']:.3f} | PESQ: {scores['PESQ']:.3f}")

        # Display Audio
        display(Audio(output_fit.cpu().detach().squeeze().numpy(), rate=sample_rate))

    # EXPERIMENT 2: SPEECH SUPER-RESOLUTION
    # Params: Conservative Noise (S0=2000, S1=0.001) to prevent overfitting.
    print(f"\n[Task 2] Super-Resolution (x{SR_FACTOR})")

    # Create Low-Res Data (Slicing)
    coords_train_sr = coords_full[::SR_FACTOR]
    waveform_train_sr = waveform_gt[::SR_FACTOR]

    # Calculate SC from Low-Res Input (Fair/Blind)
    SC_sr = spectral_centroid(waveform_train_sr.detach().cpu().numpy())

    # We use lower noise scales for inverse problems (Super-Res/Denoising)
    # S1 is reduced from 1.0 to 0.001 to prevent high-frequency "hallucinations"
    S0_sr = 2000
    S1_sr = 0.001

    # Initialize FRESH models (Duplicate) for Super-Res
    models_sr = get_networks(config, n_channels=config.n_channels, HL_dim=config.HL_dim, SC=SC_sr, S0=S0_sr, S1=S1_sr)

    for model in models_sr:
        model_name = f"{model.__class__.__name__}"
        print(f"  > Training {model_name} on Low-Res ({sample_rate//SR_FACTOR} Hz) data...")

        # Train on LOW-RES data
        batch_size = min(coords_train_sr.shape[0], 512*512)
        train(model, coords_train_sr, waveform_train_sr, config, device, nb_epochs=config.nb_epochs, batch_size=batch_size)

        # Inference on HIGH-RES grid (Super-Resolution)
        with torch.no_grad():
            output_sr = batched_forward(model, coords_full, batch_size=65536).to(device)

        # Metrics (Compare SR Output vs Ground Truth)
        scores = calculate_metrics(output_sr, waveform_gt, sample_rate)
        print(f"    {model_name} Super-Res -> PSNR: {scores['PSNR']:.2f} | LSD: {scores['LSD']:.3f} | PESQ: {scores['PESQ']:.3f}")

        # Display Audio
        display(Audio(output_sr.cpu().detach().squeeze().numpy(), rate=sample_rate))

 PROCESSING: the_rainbow.wav

[Task 1] Audio Representation (Fitting)
  > Training SIREN on Full High-Res data...


100%|██████████| 500/500 [00:15<00:00, 32.66it/s, PSNR=36.81 dB]


    SIREN Fitting -> PSNR: 43.23 | LSD: 0.781 | PESQ: 2.175


  > Training SIREN_square on Full High-Res data...


100%|██████████| 500/500 [00:15<00:00, 31.89it/s, PSNR=52.58 dB]


...

100%|██████████| 500/500 [00:07<00:00, 67.83it/s, PSNR=40.66 dB]


    SIREN Super-Res -> PSNR: 43.61 | LSD: 0.908 | PESQ: 2.131


  > Training SIREN_square on Low-Res (11025 Hz) data...


100%|██████████| 500/500 [00:07<00:00, 68.92it/s, PSNR=67.25 dB]


    SIREN_square Super-Res -> PSNR: 29.04 | LSD: 2.267 | PESQ: 1.041
